# ERSAP-E2SAR localhost development

This notebook stands up a slice of 1 node and deploys E2SAR ERSAP segmentor/reassembly pipelines. This uses loopback to test the data exchange from segmentor and reassembler

## Preparation and overview

- Be sure to [generate a keypair for Jupyter Hub](GitHubSSH.ipynb) and register it with GitHub - the keys will be used to check out the code from private repositories, like [ERSAP-Java](https://github.com/JeffersonLab/ersap-java) and [E2SAR](https://github.com/JeffersonLab/E2SAR).
- Note that for E2SAR development and testing sender and receiver node compile/build environments will be setup via post-boot scripts ([sender](post-boot/sender.sh)) and grpc++ is installed as a tar.gz with static and dynamic libraries compiled for ubuntu22
- This does not setup the control plane node for anything, but testing a specific version - you can set which branch of UDPLBd to check out and a containerized version is built and stood up.

## Preamble

This cell must be executed whether you are creating a new slice or continuing work on the old one. If you are continuing work, you then skip the slice create section and proceed to wherever you left off.

In [32]:
#
# EDIT THIS
#
# if you want to force a site instead of using random
# Pick 'UCSD', 'SRI', 'FIU' or 'TOKY' - these sites have
# IPv4. Other sites use IPv6 management and have trouble
# retrieving git-lfs artifacts.
site_override = 'SRI'
#site_override = None

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_config/github_ecdsa'

e2sar_branch = 'release' #there is no branch called release, this just keeps this notebooks slice separate.

#base distro type - either default or docker
distro_types = ['default','docker']
distro_type = distro_types[0]

# base distro 'ubuntu' or 'rocky'
distro_name = 'ubuntu'

#base distro version, currently only for ubuntu 20,22,24. E2SAR dependencies will be 
#downloaded for the appropriate versions.
distro_version = '22'

# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
home_location = {
    'ubuntu': '/home/ubuntu',
    'rocky' : '/home/rocky'
}[distro_name]

vm_key_location = f'{home_location}/.ssh/github_ecdsa'

# which test suites in E2SAR to run (leave empty to run all)
# you can set 'unit' or 'live' to run unit or live tests only
e2sar_test_suite = ''

# name of the network connecting the nodes
net_name = 'site_bridge_net'

# url of e2sar deb. Find the appropriate version for the OS at https://github.com/JeffersonLab/E2SAR/releases
e2sar_branch = "main"
static_release_url = 'https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR' # don't need to change this
e2sar_release_artifact = 'e2sar_0.1.5_amd64.deb'
e2sar_release_ver = '0.1.5'
e2sar_release_url = static_release_url + '-' + e2sar_branch + '-' + e2sar_release_ver + "-" + distro_name + "-" + distro_version + ".04/" + e2sar_release_artifact
print(e2sar_release_url)
#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

# Using docker image for cpnode by default
distro_image = distro_type + '_' + distro_name + '_' + distro_version
cp_distro_image = distro_types[1] + '_' + distro_name + '_' + distro_version

# variable settings
slice_name = f'ERSAP-E2SAR locahost test [{e2sar_branch}] on {distro_name}'

# for each node specify IP address (assuming /24), OS image
# note that most of the keys in these dictionaries map directly
# onto parameters to add_node()
node_config = {
    'sender': {
        'ip':'192.168.0.1', 
        'image': distro_image,
        'cores': 8,
        'ram': 24,
        'disk': 100 },
}
# skip these keys as they are not part of add_node params
skip_keys = ['ip']
# this is the NIC to use
nic_model = 'NIC_Basic'
# the subnet should match IPs
subnet = IPv4Network("192.168.1.0/24")

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)


https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb


Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Token File,/home/fabric/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net
Bastion Username,srinivas_0000202712
Bastion Private Key File,/home/fabric/work/fabric_config/fabric-bastion-key
Slice Public Key File,/home/fabric/work/fabric_config/slice_key.pub
Slice Private Key File,/home/fabric/work/fabric_config/slice_key
Sites to avoid,


## Create the slice

In [33]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

ID,Name,Lease Expiration (UTC),Lease Start (UTC),Project ID,State
8a1bc512-463e-4e7e-963b-c7469c6b5dd8,"UDP LB Control Plane Testing with udplbd[develop], e2sar[e2sar-java] on ubuntu",2025-02-25 22:37:15 +0000,2025-02-12 20:22:31 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK
5210f023-b755-498a-ba3a-956056527da8,4-node U280 LB Tester Slice using ubuntu22 1,2025-02-24 01:30:08 +0000,2025-01-16 21:08:36 +0000,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca,StableOK


If your slice is already active you can skip to the 'Get Slice Details' section.

In [34]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

Available images are: ['default_centos8_stream', 'default_centos9_stream', 'default_centos_7', 'default_centos_8', 'default_debian_11', 'default_debian_12', 'default_fedora_39', 'default_fedora_40', 'default_freebsd_13_zfs', 'default_freebsd_14_zfs', 'default_kali', 'default_openbsd_7', 'default_rocky_8', 'default_rocky_9', 'default_ubuntu_20', 'default_ubuntu_22', 'default_ubuntu_24', 'docker_rocky_8', 'docker_rocky_9', 'docker_ubuntu_20', 'docker_ubuntu_22']


In [35]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

# getting a random site make take a bit of time
if not site_override:
    selected_site = fablib.get_random_site(filter_function=lambda x: x['location'][1] < lon_east
                                              and x['location'][1] > lon_west) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

# write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = selected_site
    

Selected site is SRI


In [36]:
# build a slice
slice = fablib.new_slice(name=slice_name)

# create a network
net1 = slice.add_l2network(name=net_name, subnet=subnet)

nodes = dict()
# create  nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_attribs in node_config.items():
    print(f"{node_name=} {node_attribs['ip']}")
    nodes[node_name] = slice.add_node(name=node_name, **{x: node_attribs[x] for x in node_attribs if x not in skip_keys})
    nic_interface = nodes[node_name].add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attribs['ip'])
    # postboot configuration is under 'post-boot' directory
    nodes[node_name].add_post_boot_upload_directory('post-boot','.')
    nodes[node_name].add_post_boot_execute(f'chmod +x post-boot/{node_name}.sh && ./post-boot/{node_name}.sh')

print(f'Creating a {distro_name} based slice named "{slice_name}" with nodes in {selected_site}')

# Submit the slice
slice.submit();


Retry: 8, Time: 309 sec


ID,33ad6642-f018-4800-b30c-2cd012e7b2e2
Name,ERSAP-E2SAR locahost test [main] on ubuntu
Lease Expiration (UTC),2025-02-17 19:17:04 +0000
Lease Start (UTC),2025-02-16 19:17:04 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f7c81742-8d43-47d2-8e8d-4b1c03a712c0,sender,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
8d201b28-e9ee-4c90-8129-e84e5c34411d,site_bridge_net,L2,L2Bridge,SRI,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
sender-sender_NIC_Basic_nic-p1,p1,sender,site_bridge_net,100,config,,06:13:45:4B:83:8B,enp7s0,enp7s0,192.168.0.1,6,HundredGigE0/0/0/5



Time to print interfaces 309 seconds


## Get Slice Details

If not creating a new slice, and just continuing work on an existing one, execute this cell (in addition to the preamble) and then any of the cells below will work.

In [37]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

sender = slice.get_node(name="sender")


# get node dataplane addresses
sender_addr = sender.get_interface(network_name=net_name).get_ip_addr()

sender_iface = sender.get_interface(network_name=net_name)

ID,33ad6642-f018-4800-b30c-2cd012e7b2e2
Name,ERSAP-E2SAR locahost test [main] on ubuntu
Lease Expiration (UTC),2025-02-17 19:17:04 +0000
Lease Start (UTC),2025-02-16 19:17:04 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
8d201b28-e9ee-4c90-8129-e84e5c34411d,site_bridge_net,L2,L2Bridge,SRI,192.168.1.0/24,None,Active,


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
f7c81742-8d43-47d2-8e8d-4b1c03a712c0,sender,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.168,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.168,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


## Download and install E2SAR deb

In [38]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"echo 'export LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.profile",
    f"echo 'export LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.bashrc",
]

for node in [sender]:    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

	Executing "chmod go-rwx /home/ubuntu/.ssh/github_ecdsa" on node sender
	Executing "echo 'export LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.profile" on node sender
	Executing "echo 'export LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64' >> ~/.bashrc" on node sender


In [39]:
#set JAVA_HOME enviroment variables
java_home = '/usr/lib/jvm/java-1.17.0-openjdk-amd64' ## probably have to change this depending on OS. Tested on Ubuntu 22.04
commands = [
    f"echo 'export JAVA_HOME={java_home}' >> ~/.profile",
    f"echo 'export JAVA_HOME={java_home}' >> ~/.bashrc",
]
 
execute_commands(sender, commands)

	Executing "echo 'export JAVA_HOME=/usr/lib/jvm/java-1.17.0-openjdk-amd64' >> ~/.profile" on node sender
	Executing "echo 'export JAVA_HOME=/usr/lib/jvm/java-1.17.0-openjdk-amd64' >> ~/.bashrc" on node sender


In [40]:
#download boost and grpc dependencies from releases
commands = [
    f"wget -q -O e2sar-release.deb {e2sar_release_url}",
    f"sudo apt -yq install ./e2sar-release.deb",
]
 
execute_commands([sender], commands)

	Executing "wget -q -O e2sar-release.deb https://github.com/JeffersonLab/E2SAR/releases/download/E2SAR-main-0.1.5-ubuntu-22.04/e2sar_0.1.5_amd64.deb" on node sender
	Executing "sudo apt -yq install ./e2sar-release.deb" on node sender
Reading package lists... 

 
Building dependency tree...
Reading state information...
The following NEW packages will be installed:
  e2sar
0 upgraded, 1 newly installed, 0 to remove and 29 not upgraded.
Need to get 0 B/85.2 MB of archives.
After this operation, 465 MB of additional disk space will be used.
Get:1 /home/ubuntu/e2sar-release.deb e2sar amd64 0.1.5 [85.2 MB]
 debconf: unable to initialize frontend: Dialog
debconf: (Dialog frontend will not work on a dumb terminal, an emacs shell buffer, or without a controlling terminal.)
debconf: falling back to frontend: Readline
  debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-o

# Downloading ERSAP and E2SAR java libraries

In [41]:
ersap_home = '$HOME/ersap-install'
commands = [
    f"echo 'export ERSAP_HOME={ersap_home}' >> ~/.profile",
    f"echo 'export ERSAP_HOME={ersap_home}' >> ~/.bashrc",
]
execute_commands([sender], commands)

	Executing "echo 'export ERSAP_HOME=$HOME/ersap-install' >> ~/.profile" on node sender
	Executing "echo 'export ERSAP_HOME=$HOME/ersap-install' >> ~/.bashrc" on node sender


In [42]:
commands = [
    f"mkdir ERSAP",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b upgradeGradle git@github.com:JeffersonLab/ersap-java.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-e2sar.git",
    f"cd ERSAP; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/e2sar-java.git",
]
execute_commands([sender], commands)

	Executing "mkdir ERSAP" on node sender
	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b upgradeGradle git@github.com:JeffersonLab/ersap-java.git" on node sender
 Cloning into 'ersap-java'...
 	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/ersap-e2sar.git" on node sender
 Cloning into 'ersap-e2sar'...
 	Executing "cd ERSAP; GIT_SSH_COMMAND='ssh -i /home/ubuntu/.ssh/github_ecdsa -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b main git@github.com:JeffersonLab/e2sar-java.git" on node sender
 Cloning into 'e2sar-java'...
 

## Installing erap-java
ERSAP_HOME for this test is $HOME/ersap-install

In [ ]:
commands = [ 
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-java; ./gradlew deploy",
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-java; ./gradlew publishToMavenLocal",
]
execute_commands([sender], commands)

In [44]:
commands = [ 
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-cpp; ./configure --prefix='$ERSAP_HOME'",
    f"cd ERSAP/ersap-cpp; make install"
]
execute_commands([sender], commands)

	Executing "export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-cpp; ./configure --prefix='$ERSAP_HOME'" on node sender
 bash: line 1: cd: ERSAP/ersap-cpp: No such file or directory
bash: line 1: ./configure: No such file or directory
 	Executing "cd ERSAP/ersap-cpp; make install" on node sender
 bash: line 1: cd: ERSAP/ersap-cpp: No such file or directory
make: *** No rule to make target 'install'.  Stop.
 

## Building and installing E2SAR-JAVA

In [48]:
#Building the JNI Shared library
#Might have to set PKG_CONFIG_PATH for cmake build to work. Works without pkg_config_path on 22.04
commands = [
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} JAVA_HOME={java_home} cmake -DCMAKE_INSTALL_PREFIX={ersap_home}/plugins/jni/ -S . -B build",
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} JAVA_HOME={java_home} sudo cmake --build build --target install",
]

execute_commands(sender, commands)

	Executing "cd ERSAP/e2sar-java; ERSAP_HOME=$HOME/ersap-install JAVA_HOME=/usr/lib/jvm/java-1.17.0-openjdk-amd64 cmake -DCMAKE_INSTALL_PREFIX=$HOME/ersap-install/plugins/jni/ -S . -B build" on node sender
-- JNI_INCLUDE_DIRS=/usr/lib/jvm/java-1.17.0-openjdk-amd64/include;/usr/lib/jvm/java-1.17.0-openjdk-amd64/include/linux;/usr/lib/jvm/java-1.17.0-openjdk-amd64/include
-- JNI_LIBRARIES=/usr/lib/jvm/java-1.17.0-openjdk-amd64/lib/libjawt.so;/usr/lib/jvm/java-1.17.0-openjdk-amd64/lib/server/libjvm.so
-- E2SAR_INCLUDE_DIRS: /usr/local/include
-- E2SAR linker flags: e2sar
-- GRPC linker flags: grpc++;grpc;gpr;absl_flags;absl_flags_internal;absl_flags_reflection;absl_flags_private_handle_accessor;absl_flags_commandlineflag;absl_flags_commandlineflag_internal;absl_flags_config;absl_flags_program_name;absl_flags_marshalling;absl_raw_hash_set;absl_hashtablez_sampler;absl_hash;absl_city;absl_low_level_hash;absl_random_distributions;absl_random_seed_sequences;absl_random_internal_pool_urbg;absl_r

In [ ]:
#Compiling Java code and packaging into jar
##IMPORTANT MAVEN ARGUMENTS NEEDED FOR TESTS
# Need to specify -Djava.library.path for linking with the jnie2sar.so built in the step above. Maven surefire tests do not pick this up directly so it has to be encapsulated as -DargLine='-Djava.library.path=build/'
# -Dtest="" is needed to specify class/package of tests with pattern. If this is not specified then all classes with *TEST* will be run by maven surefire

commands = [
    f"cd ERSAP/e2sar-java; ERSAP_HOME={ersap_home} LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64 mvn -DargLine='-Djava.library.path=${{env.ERSAP_HOME}}/plugins/jni/lib' clean install -Dtest='org.jlab.hpdf.unit.**'",
]

execute_commands([sender], commands)

In [55]:
##Copying build jar to ERSAP_HOME/lib 
commands = [
    f"export ERSAP_HOME={ersap_home}; cp ~/.m2/repository/org/jlab/hpdf/e2sar-java/0.0.1/e2sar-java-0.0.1-jar-with-dependencies.jar $ERSAP_HOME/lib ",
]

execute_commands([sender], commands)


	Executing "export ERSAP_HOME=$HOME/ersap-install; cp ~/.m2/repository/org/jlab/hpdf/e2sar-java/0.0.1/e2sar-java-0.0.1-jar-with-dependencies.jar $ERSAP_HOME/lib " on node sender


## Installing ersap-e2sar-java (Pure JAVA pipeline with JNI for E2SAR)

In [50]:
commands = [ 
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-e2sar/e2sar-java-actors/segmentorJni; ./gradlew install",
    f"export ERSAP_HOME={ersap_home}; cd ERSAP/ersap-e2sar/e2sar-java-actors/reassemblerJni; ./gradlew install",
]
execute_commands([sender], commands)

	Executing "export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/segmentorJni; ./gradlew install" on node sender
> Task :compileJava
> Task :processResources
> Task :classes
> Task :jar
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

> Task :installJar
JAR installed to/home/ubuntu/ersap-install/lib

BUILD SUCCESSFUL in 1s
4 actionable tasks: 4 executed
	Executing "export ERSAP_HOME=$HOME/ersap-install; cd ERSAP/ersap-e2sar/reassemblerJni; ./gradlew install" on node sender
> Task :compileJava
> Task :processResources
> Task :classes
> Task :jar
> Task :assemble
> Task :compileTestJava NO-SOURCE
> Task :processTestResources NO-SOURCE
> Task :testClasses UP-TO-DATE
> Task :test NO-SOURCE
> Task :check UP-TO-DATE
> Task :build

> Task :installJar
JAR installed to/home/ubuntu/ersap-install/lib

BUILD SUCCESSFUL in 845ms
4 actionable tasks: 4 executed

## Running reassembler and segmentor localhost actors

In [51]:
# deleting the output file in reassembler to make sure this works. Also printing out the input file

commands = [ 
    f"rm ERSAP/ersap-e2sar/reassemblerJni/output/out_in.txt",
    f"echo 'input events:';cat ERSAP/ersap-e2sar/segmentorJni/input/in.txt",
]
execute_commands([sender], commands)

	Executing "rm ERSAP/ersap-e2sar/reassemblerJni/output/out_in.txt" on node sender
	Executing "echo 'input events:';cat ERSAP/ersap-e2sar/segmentorJni/input/in.txt" on node sender
input events:
2
Event 1: This is a sample input for my ersap-e2sar test
Event 2: This is my second input, hopefully I can read both of them and pass it to the segmenter. This is also a longer input so should get more data frames

In [57]:
import time
recv_command = f"export ERSAP_HOME={ersap_home} LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64; cd ERSAP/ersap-e2sar/e2sar-java-actors/reassemblerJni; $HOME/ersap-install/bin/ersap-shell config/localhost/reassembler_localhost.ersap"
send_command = f"export ERSAP_HOME={ersap_home} LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64; cd ERSAP/ersap-e2sar/e2sar-java-actors/segmentorJni; $HOME/ersap-install/bin/ersap-shell config/localhost/segmentor_localhost.ersap"

# start the receiver for 10 seconds and log its output
print(f'Executing command {recv_command} on sender-reassembler')
sender.execute_thread(recv_command, output_file=f"reassembler-localhost.ersap-e2sar.log")

time.sleep(1)

# start the sender in the foreground
print(f'Executing command {send_command} on sender-segmentor')
stdout_send, stderr_send = sender.execute(send_command, output_file=f"{sender.get_name()}.perf.log")

print(f"Inspect reassembler-localhost.ersap-e2sar.log file in your Jupyter container to see the results")

Executing command export ERSAP_HOME=$HOME/ersap-install LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64; cd ERSAP/ersap-e2sar/reassemblerJni; $HOME/ersap-install/bin/ersap-shell config/localhost/reassembler_localhost.ersap on sender-reassembler
Executing command export ERSAP_HOME=$HOME/ersap-install LD_LIBRARY_PATH=/usr/local/lib:/usr/local/lib64; cd ERSAP/ersap-e2sar/segmentorJni; $HOME/ersap-install/bin/ersap-shell config/localhost/segmentor_localhost.ersap on sender-segmentor
         It will be set to point to the ERSAP_HOME/plugins/epsci.
         Note that you might face permission exceptions. 
 Feb 16, 2025 7:37:06 PM org.jline.utils.Log logr
  tee: /home/ubuntu/ersap-install/plugins/epsci/log/10.20.5.231_ubuntu_ersap_fe_dpe.log: No such file or directory
 OpenJDK 64-Bit Server VM warning: Option UseBiasedLocking was deprecated in version 15.0 and will likely be removed in a future release.

               ERSAP FE/DPE               
 Name             = 10.20.5.231%8400_java
 Se

In [58]:
print("Output file contents")
# deleting the output file in reassembler to make sure this works. Also printing out the input file

commands = [ 
    f"cat ERSAP/ersap-e2sar/reassemblerJni/output/out_in.txt",
]
execute_commands([sender], commands)

Output file contents
	Executing "cat ERSAP/ersap-e2sar/reassemblerJni/output/out_in.txt" on node sender
Event 1: This is a sample input for my ersap-e2sar test
Event 2: This is my second input, hopefully I can read both of them and pass it to the segmenter. This is also a longer input so should get more data frames


## Manage the slice

### Extend

In [4]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")


Retry: 0, Time: 23 sec


ID,c390ffe9-cdb7-4222-bbe5-7c3f2fc4c498
Name,ERSAP-E2SAR locahost test [main] on ubuntu
Lease Expiration (UTC),2025-02-28 15:04:14 +0000
Lease Start (UTC),2025-02-14 20:08:44 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
af8090c4-19c4-4369-b9dc-08aab0cdfd12,sender,8,32,100,default_ubuntu_22,qcow2,sri-w1.fabric-testbed.net,SRI,ubuntu,192.5.67.30,Active,,ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@192.5.67.30,/home/fabric/work/fabric_config/slice_key.pub,/home/fabric/work/fabric_config/slice_key


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
44fe0f29-b822-4820-8c99-2a191a70c9d7,site_bridge_net,L2,L2Bridge,SRI,192.168.1.0/24,None,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
sender-sender_NIC_Basic_nic-p1,p1,sender,site_bridge_net,100,config,,06:13:45:4B:83:8B,enp7s0,enp7s0,192.168.0.1,6,HundredGigE0/0/0/5



Time to print interfaces 23 seconds


### Delete

In [11]:
slice = fablib.get_slice(slice_name)
slice.delete()